**Faiss 是 Facebook AI Search 釋出的開源程式庫，不算是資料庫，概念是針對向量建立索引並進行高效率搜索。**

**Faiss的安裝,一般使用cpu版本就足夠**
```
pip install faiss-cpu
pip install faiss-gpu
```

### Step1 載入和準備數據集

In [47]:
from datasets import load_dataset
dataset = load_dataset('roberthsu2003/data_for_RAG',split='train')
#增加一個columns,`text`
dataset = dataset.map(lambda x:{'text':
                      'passage:' + x['question'] + '\n' + x['answering']
                      })

In [48]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

In [49]:
#將modle,設定至cpu或gpu上執行
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.device

device(type='cpu')

In [51]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt", max_length=512
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    with torch.no_grad():
        model_output = model(**encoded_input)
        embeddings = model_output.last_hidden_state[:, 0, :]
        #embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
    return embeddings

In [52]:
embedding_dataset = dataset.map(
    lambda x:{"embeddings":get_embeddings(x['text']).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/123 [00:00<?, ? examples/s]

In [53]:
#使用FAISS進行高效的相似性搜索
#Dataset.add_faiss_index()
embedding_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['question', 'answering', 'text', 'embeddings'],
    num_rows: 123
})

In [90]:
question = "充電埠在哪裡？"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 1024)

In [91]:
scores, samples = embedding_dataset.get_nearest_examples(
    "embeddings", question_embedding, k = 5
)

In [92]:
import pandas as pd
samples_df = pd.DataFrame.from_dict(samples)
samples_df['scores'] = scores
samples_df.sort_values('scores', ascending=True, inplace=True)

In [93]:
for _, row in samples_df.iterrows():
    print(f"question:",{row.question})
    print(f"answering:",{row.answering})
    print('score:',{row.scores})
    print("="*50)
    print()

question: {'Model 3 的充電埠在哪裡？'}
answering: {'充電埠位於車輛左後方，整合在尾燈組的一個蓋板後面。[source: 327]'}
score: {238.14675903320312}

question: {'如何使用 Model 3 的無線手機充電器？'}
answering: {'將支援 Qi 功能的手機放置在前中控台的兩個充電板之一即可充電。手機需直接接觸充電板，充電前請移除手機與充電器間的任何物體（金屬、NFC 卡等）。[source: 334]'}
score: {298.406982421875}

question: {'哪個 USB 連接埠最適合用於儲存哨兵模式和行車記錄器影像？'}
answering: {'位於手套箱內的 USB-A 連接埠最適合儲存哨兵模式和行車記錄器影像，因為位置較安全。[source: 334]'}
score: {302.7074279785156}

question: {'Model 3 有多少個 USB 連接埠？分別在哪裡？'}
answering: {'Model 3 共有四個 USB 連接埠。一個 USB-C 在中控台後方置物空間，兩個 USB-C 在中控台後車廂出風口下方，一個 USB-A 在手套箱內。[source: 334]'}
score: {306.719482421875}

question: {'如何使用卡片鑰匙解鎖或上鎖 Model 3？'}
answering: {'將卡片鑰匙放置在駕駛座側門柱 B 柱上大約三分之一位置的讀卡機感應。聽到提示音、看到車燈閃爍或後視鏡動作，即表示完成上鎖或解鎖。[source: 342]'}
score: {313.57861328125}

